In [1]:
import json

f = open('data/vectorized_lines.json', 'r', encoding='utf-8')
vectorized_lines = json.load(f)
f.close()

print(len(vectorized_lines))
print(vectorized_lines[0])

58488
{'line_text': ['PAinter', ',', 'once', 'more', 'thy', 'Pencil', 'reassume', '.'], 'line_pos': ['n1', '', 'acp', 'avc-d', 'po', 'n1', 'vvi', ''], 'line_arpabet': ['p/ey/n/t/er', '', 'w/ah/n/s', 'm/ao/r', 'dh/ay', 'p/eh/n/s/ah/l', 'r/iy/ah/s/uw/m', ''], 'line_ipa': ['p/eɪ/n/t/ɹ̩', '', 'w/ʌ/n/s', 'm/ɔ/ɹ', 'ð/aɪ', 'p/ɛ/n/s/ʌ/l', 'ɹ/i/ʌ/s/u/m', ''], 'line_hayes_features': ['001000100000000000000000000000/010010001001010000001000101010/100100100000000000101000000010/100100100000000000000000000000/100010001010000000001000100010', '', '011011001000100000011000001010/010011001000000000001000100010/100100100000000000101000000010/100100101100000000000001000000', '001000100000000000101000000010/011011001000000000011000100010/100010001010000000001000000010', '100100101110000000000000000010/010010001000010001001000100010', '001000100000000000000000000000/010010001001000000001000100010/100100100000000000101000000010/100100101100000000000001000000/010011001000000000001000100010/10011010100000010

In [2]:
lines_by_author = {}

for l in vectorized_lines:
    if l['author'] not in lines_by_author:
        lines_by_author[l['author']] = []
    lines_by_author[l['author']].append(l)
    
for author, lines in lines_by_author.items():
    print(author, len(lines))
    
author_names = sorted(list(lines_by_author.keys()))

print()
print(author_names)

Marvell 8418
Vaughan 8893
Donne 13346
Crashaw 8545
Rochester 3875
Herbert 5420
Herrick 9991

['Crashaw', 'Donne', 'Herbert', 'Herrick', 'Marvell', 'Rochester', 'Vaughan']


In [9]:
import random

import numpy as np
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

classifier_names = [
    "Nearest Neighbors",
    "Linear SVM",
    "RBF SVM",
    "Gaussian Process",
    "Decision Tree",
    "Random Forest",
    "Neural Net",
    "AdaBoost",
    "Naive Bayes",
    "QDA",
]

classifiers = [
    KNeighborsClassifier(2),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis(),
]
        
# -----------------------------------------------------------------------------------

def lines_to_features(lines):
    
    all_lines_features = []
    
    for l in lines:
        
        l_features = [0 for a in range(0, 30)]
        
        hayes = '/'.join(l['line_hayes_features']).split('/')
        
        for h in hayes:
            i_features = [int(c) for c in h]
            for a, i in enumerate(i_features):
                l_features[a] += i
                
        all_lines_features.append(np.array(l_features))

    return np.array(all_lines_features)
        
# -----------------------------------------------------------------------------------
        
for a in range(0, len(author_names) - 1):
    for b in range(a + 1, len(author_names)):
        
        a_lines = lines_by_author[author_names[a]]
        b_lines = lines_by_author[author_names[b]]
        
        random.shuffle(a_lines)
        random.shuffle(b_lines)
        
        lines_to_features([a_lines[0]])
        
        a_training = lines_to_features(a_lines[:3000])
        a_testing = lines_to_features(a_lines[3000:])
        
        b_training = lines_to_features(b_lines[:3000])
        b_testing = lines_to_features(b_lines[3000:])
        
        low_testing_n = len(a_testing)
        if len(b_testing) < low_testing_n:
            low_testing_n = len(b_testing)
            
        a_testing = a_testing[:low_testing_n]
        b_testing = b_testing[:low_testing_n]
        
        
        print()
        print(author_names[a], 'vs.', author_names[b])
        print()
        
        print('\t', 'a_lines', len(a_lines), 'b_lines', len(b_lines))
        print()
        print('\t', 'a_training', len(a_training))
        print('\t', 'a_testing', len(a_testing))
        print()
        print('\t', 'b_training', len(b_training))
        print('\t', 'b_testing', len(b_testing))
        print()
        
        # -----------------------------------------------------------------------------------
        
        for classifier_name, classifier in zip(classifier_names, classifiers):
            
            classifier.fit(a_training, b_training)
            score = classifier.score(a_testing, b_testing)
            
            print('\t', classifier_name, score)
            
            
        
        break
    break


Crashaw vs. Donne

	 a_lines 8545 b_lines 13346

	 a_training 3000
	 a_testing 5545

	 b_training 3000
	 b_testing 5545



ValueError: bad input shape (3000, 30)